In [1]:
"""
Please run notebook locally (if you have all the dependencies and a GPU). 
Technically you can run this notebook on Google Colab but you need to set up microphone for Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Set up microphone for Colab
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
#!pip install wget
#!apt-get install sox libsndfile1 ffmpeg portaudio19-dev
#!pip install unidecode
#!pip install pyaudio

# ## Install NeMo
#BRANCH = 'main'
#!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]

## Install TorchAudio
#!pip install torchaudio>=0.6.0 -f https://download.pytorch.org/whl/torch_stable.html

## Grab the config we'll use in this example
#!mkdir configs

This notebook demonstrates automatic speech recognition (ASR) from a microphone's stream in NeMo.

It is **not a recommended** way to do inference in production workflows. If you are interested in 
production-level inference using NeMo ASR models, please refer to NVIDIA RIVA: https://developer.nvidia.com/riva

The notebook requires PyAudio library to get a signal from an audio device.
For Ubuntu, please run the following commands to install it:
```
sudo apt-get install -y portaudio19-dev
pip install pyaudio
```

This notebook requires the `torchaudio` library to be installed for MatchboxNet. Please follow the instructions available at the [torchaudio Github page](https://github.com/pytorch/audio#installation) to install the appropriate version of torchaudio.

If you would like to install the latest version, please run the following command to install it:

```
conda install -c pytorch torchaudio
```

In [2]:
import numpy as np
import pyaudio as pa
import os, time

In [3]:
import nemo
import nemo.collections.asr as nemo_asr

D:\anaconda3\lib\site-packages\pytorch_lightning\metrics\__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(
[NeMo W 2021-09-25 00:22:20 optimizers:47] Apex was not found. Using the lamb optimizer will error out.
[NeMo W 2021-09-25 00:22:23 nemo_logging:349] D:\anaconda3\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
      warnings.warn('torchaudio C++ extension is not available.')
    
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
##############

In [4]:
# sample rate, Hz
SAMPLE_RATE = 16000

In [5]:
asr_model = nemo_asr.models.EncDecCTCModel.restore_from(r'D:\work\am\QuartzNet15x5_golos.nemo')

[NeMo W 2021-09-25 00:22:28 modelPT:138] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2021-09-25 00:22:28 modelPT:145] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2021-09-25 00:22:28 features:252] PADDING: 16
[NeMo I 2021-09-25 00:22:28 features:269] STFT using torch
[NeMo I 2021-09-25 00:22:30 modelPT:438] Model EncDecCTCModel was successfully restored from D:\work\am\QuartzNet15x5_golos.nemo.


## Restore the model from NGC

In [6]:
#asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained('QuartzNet15x5Base-En')

## Observing the config of the model

In [7]:
from omegaconf import OmegaConf
import copy

In [8]:
# Preserve a copy of the full config
cfg = copy.deepcopy(asr_model._cfg)
#print(OmegaConf.to_yaml(cfg))

### Modify preprocessor parameters for inference

In [9]:
# # Make config overwrite-able
# OmegaConf.set_struct(cfg.preprocessor, False)

# # some changes for streaming scenario
# cfg.preprocessor.dither = 0.0
# cfg.preprocessor.pad_to = 0

# # spectrogram normalization constants
# normalization = {}
# normalization['fixed_mean'] = [
#      -14.95827016, -12.71798736, -11.76067913, -10.83311182,
#      -10.6746914,  -10.15163465, -10.05378331, -9.53918999,
#      -9.41858904,  -9.23382904,  -9.46470918,  -9.56037,
#      -9.57434245,  -9.47498732,  -9.7635205,   -10.08113074,
#      -10.05454561, -9.81112681,  -9.68673603,  -9.83652977,
#      -9.90046248,  -9.85404766,  -9.92560366,  -9.95440354,
#      -10.17162966, -9.90102482,  -9.47471025,  -9.54416855,
#      -10.07109475, -9.98249912,  -9.74359465,  -9.55632283,
#      -9.23399915,  -9.36487649,  -9.81791084,  -9.56799225,
#      -9.70630899,  -9.85148006,  -9.8594418,   -10.01378735,
#      -9.98505315,  -9.62016094,  -10.342285,   -10.41070709,
#      -10.10687659, -10.14536695, -10.30828702, -10.23542833,
#      -10.88546868, -11.31723646, -11.46087382, -11.54877829,
#      -11.62400934, -11.92190509, -12.14063815, -11.65130117,
#      -11.58308531, -12.22214663, -12.42927197, -12.58039805,
#      -13.10098969, -13.14345864, -13.31835645, -14.47345634]
# normalization['fixed_std'] = [
#      3.81402054, 4.12647781, 4.05007065, 3.87790987,
#      3.74721178, 3.68377423, 3.69344,    3.54001005,
#      3.59530412, 3.63752368, 3.62826417, 3.56488469,
#      3.53740577, 3.68313898, 3.67138151, 3.55707266,
#      3.54919572, 3.55721289, 3.56723346, 3.46029304,
#      3.44119672, 3.49030548, 3.39328435, 3.28244406,
#      3.28001423, 3.26744937, 3.46692348, 3.35378948,
#      2.96330901, 2.97663111, 3.04575148, 2.89717604,
#      2.95659301, 2.90181116, 2.7111687,  2.93041291,
#      2.86647897, 2.73473181, 2.71495654, 2.75543763,
#      2.79174615, 2.96076456, 2.57376336, 2.68789782,
#      2.90930817, 2.90412004, 2.76187531, 2.89905006,
#      2.65896173, 2.81032176, 2.87769857, 2.84665271,
#      2.80863137, 2.80707634, 2.83752184, 3.01914511,
#      2.92046439, 2.78461139, 2.90034605, 2.94599508,
#      2.99099718, 3.0167554,  3.04649716, 2.94116777]

# cfg.preprocessor.normalize = normalization

# # Disable config overwriting
# OmegaConf.set_struct(cfg.preprocessor, True)

## Setup preprocessor with these settings

In [10]:
#asr_model.preprocessor = asr_model.from_config_dict(cfg.preprocessor)

In [11]:
# Set model to inference mode
asr_model.eval();

In [12]:
asr_model = asr_model.to(asr_model.device)

## Setting up data for Streaming Inference

In [13]:
from nemo.core.classes import IterableDataset
from nemo.core.neural_types import NeuralType, AudioSignal, LengthsType
import torch
from torch.utils.data import DataLoader

In [14]:
# simple data layer to pass audio signal
class AudioDataLayer(IterableDataset):
    @property
    def output_types(self):
        return {
            'audio_signal': NeuralType(('B', 'T'), AudioSignal(freq=self._sample_rate)),
            'a_sig_length': NeuralType(tuple('B'), LengthsType()),
        }

    def __init__(self, sample_rate):
        super().__init__()
        self._sample_rate = sample_rate
        self.output = True
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if not self.output:
            raise StopIteration
        self.output = False
        return torch.as_tensor(self.signal, dtype=torch.float32), \
               torch.as_tensor(self.signal_shape, dtype=torch.int64)
        
    def set_signal(self, signal):
        self.signal = signal.astype(np.float32)/32768.
        self.signal_shape = self.signal.size
        self.output = True

    def __len__(self):
        return 1

In [15]:
data_layer = AudioDataLayer(sample_rate=16000)
data_loader = DataLoader(data_layer, batch_size=1, collate_fn=data_layer.collate_fn)

In [16]:
# inference method for audio signal (single instance)
def infer_signal(model, signal):
    data_layer.set_signal(signal)
    batch = next(iter(data_loader))
    audio_signal, audio_signal_len = batch
    audio_signal, audio_signal_len = audio_signal.to(asr_model.device), audio_signal_len.to(asr_model.device)
    log_probs, encoded_len, predictions = model.forward(
        input_signal=audio_signal, input_signal_length=audio_signal_len
    )
    return log_probs

In [17]:
from demo_decoder import CTCBeamSearchDecoder

# class for streaming frame-based ASR
# 1) use reset() method to reset FrameASR's state
# 2) call transcribe(frame) to do ASR on
#    contiguous signal's frames
class FrameASR:
    
    def __init__(self, model_definition,
                 kenlm_model,
                 frame_len=2, frame_overlap=2.5, 
                 show_fixed_part=True,
                 offset=10):
        '''
        Args:
          frame_len: frame's duration, seconds
          frame_overlap: duration of overlaps before and after current frame, seconds
          offset: number of symbols to drop for smooth streaming
        '''
        self.vocab = list(model_definition['labels'])
        self.vocab.append('_')
        
        self.sr = model_definition['sample_rate']
        self.frame_len = frame_len
        self.n_frame_len = int(frame_len * self.sr)
        self.frame_overlap = frame_overlap
        self.n_frame_overlap = int(frame_overlap * self.sr)
        timestep_duration = model_definition['AudioToMelSpectrogramPreprocessor']['window_stride']
        for block in model_definition['JasperEncoder']['jasper']:
            timestep_duration *= block['stride'][0] ** block['repeat']
        self.n_timesteps_overlap = int(frame_overlap / timestep_duration) - 2
        self.buffer = np.zeros(shape=2*self.n_frame_overlap + self.n_frame_len,
                               dtype=np.float32)
        self.offset = offset
        self.reset()

        # Decoder
        self.decoder = CTCBeamSearchDecoder(kenlm_model, self.vocab,
                                            beam_size=15, loglike_gap=3.0,
                                            acoustic_loglike_gap=2.0,
                                            detach_fixed_part=True)
        self.decoder.reset()

    @torch.no_grad()
    def _decode(self, frame, offset=0):
        #print(frame, len(frame))
        assert len(frame) == self.n_frame_len
        self.buffer[:-self.n_frame_len] = self.buffer[self.n_frame_len:]
        self.buffer[-self.n_frame_len:] = frame
        logits = infer_signal(asr_model, self.buffer).cpu().numpy()[0]
        #decoded = self._greedy_decoder(
        #    logits[self.n_timesteps_overlap:-self.n_timesteps_overlap],
        #    self.vocab
        #)
        for frame_logits in logits:
            self.decoder.process_frame(frame_logits)
        return self.decoder.tree()
    
    @torch.no_grad()
    def transcribe(self, frame=None, merge=True):
        if frame is None:
            frame = np.zeros(shape=self.n_frame_len, dtype=np.float32)
        if len(frame) < self.n_frame_len:
            frame = np.pad(frame, [0, self.n_frame_len - len(frame)], 'constant')
        unmerged = self._decode(frame, self.offset)
        if not merge:
            return unmerged
        return self.greedy_merge(unmerged)
    
    def reset(self):
        '''
        Reset frame_history and decoder's state
        '''
        self.buffer=np.zeros(shape=self.buffer.shape, dtype=np.float32)
        self.prev_char = ''
        
    def best_hyp(self):
        return self.decoder.best_hyp()

    @staticmethod
    def _greedy_decoder(logits, vocab):
        s = ''
        for i in range(logits.shape[0]):
            s += vocab[np.argmax(logits[i])]
        return s

    def greedy_merge(self, s):
        s_merged = ''
        
        for i in range(len(s)):
            if s[i] != self.prev_char:
                self.prev_char = s[i]
                if self.prev_char != '_':
                    s_merged += self.prev_char
        return s_merged

# Streaming Inference

Streaming inference depends on a few factors, such as the frame length and buffer size. Experiment with a few values to see their effects in the below cells.

In [18]:
from IPython.display import display, clear_output

# duration of signal frame, seconds
FRAME_LEN = 1.0
# number of audio channels (expect mono signal)
CHANNELS = 1

import kenlm
lm_golos = kenlm.Model(r'D:\work\am\lm_golos.binary')

CHUNK_SIZE = int(FRAME_LEN*SAMPLE_RATE)
asr = FrameASR(model_definition = {
                   'sample_rate': SAMPLE_RATE,
                   'AudioToMelSpectrogramPreprocessor': cfg.preprocessor,
                   'JasperEncoder': cfg.encoder,
                   'labels': cfg.decoder.vocabulary
               },
               kenlm_model=lm_golos,
               frame_len=FRAME_LEN, frame_overlap=1,
               offset=0)

In [19]:
# asr.reset()

p = pa.PyAudio()
#print('Available audio input devices:')
input_devices = []
for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    if dev.get('maxInputChannels'):
        input_devices.append(i)
        print(i, dev.get('name'))

if len(input_devices):
    dev_idx = -2
    while dev_idx not in input_devices:
        print('Please type input device ID:')
        dev_idx = int(input())

    empty_counter = 0

    def callback(in_data, frame_count, time_info, status):
        global empty_counter
        signal = np.frombuffer(in_data, dtype=np.int16)
        tree = asr._decode(signal)
        clear_output(wait=True)
        display(asr.best_hyp())
        return (in_data, pa.paContinue)

    stream = p.open(format=pa.paInt16,
                    channels=CHANNELS,
                    rate=SAMPLE_RATE,
                    input=True,
                    input_device_index=dev_idx,
                    stream_callback=callback,
                    frames_per_buffer=CHUNK_SIZE)

    print('Listening...')

    stream.start_stream()
    
    # Interrupt kernel and then speak for a few more words to exit the pyaudio loop !
    try:
        while stream.is_active():
            time.sleep(0.1)
    finally:        
        stream.stop_stream()
        stream.close()
        p.terminate()

        print()
        print("PyAudio stopped")
    
else:
    print('ERROR: No audio input device found.')

0 Microsoft Sound Mapper - Input
1 Headset (FreeBuds 3 Hands-Free 
2 Microphone (Realtek(R) Audio)
7 Stereo Mix (Realtek HD Audio Stereo input)
10 PC Speaker (Realtek HD Audio output with HAP)
11 Microphone (Realtek HD Audio Mic input)
15 Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(FreeBuds 3))
17 Input (@System32\drivers\bthhfenum.sys,#4;%1 Hands-Free HF Audio%0
;(HUAWEI MediaPad M5 lite 10))
18 Line ()
20 Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(HUAWEI CM510))
22 Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(HUAWEI CM510))
Please type input device ID:


 1


Listening...


[NeMo W 2021-09-25 00:22:35 patch_utils:49] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.
[NeMo W 2021-09-25 00:22:35 nemo_logging:349] D:\anaconda3\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
    To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ..\aten\src\ATen\native\BinaryOps.cpp:467.)
      return torch.floor_divide(self, other)
    


TypeError: 'tuple' object does not support item assignment


PyAudio stopped


TypeError: 